In [1]:
# GLOBAL VARIABLES. BE SURE NOT TO OVERWRITE THEM
D = 5 # Amount of documents
V = 10 # Size of the vocabulary

# Maximum allowed amount of same word repetition in a document (it may be lower in practise due to the data generation strategy)
# To get the actual maximum amount, call Simulator.get_M
M = 10 

k = 5 # Amount of topics
gamma = 0.05

In [2]:
# # GLOBAL VARIABLES. BE SURE NOT TO OVERWRITE THEM
# D = 100 # Amount of documents
# V = 20 # Size of the vocabulary

# # Maximum allowed amount of same word repetition in a document (it may be lower in practise due to the data generation strategy)
# # To get the actual maximum amount, call Simulator.get_M
# M = 10 

# k = 10 # Amount of topics
# gamma = 0.05

## IMPORTANT: Please use static random seeds in **EVERY** cell where you use a random function, so that the result does **NOT** change at every run.

# 1. ARTIFICIAL DATA

### Task:

You must implement an algorithm that generates an artificial *corpus*, and return also a graph G and a correlation matrix Sigma.

In [3]:
import numpy as np
from scipy.stats import bernoulli
#!pip install sklearn
from sklearn.datasets import make_sparse_spd_matrix

In [4]:
# Transformation functions (deterministic)

def update_Theta(Theta, H):
    for d in range(D):
        Theta[d] = np.exp(H[d]) / np.sum(np.exp(H[d]), axis=0)
    print('Success: Theta transformed from H')
    return Theta

def update_E(E, Z):
    k = E.shape[1]
    for topic in range(k):
        E[:, topic] = np.sum(Z == topic, axis=2).sum(axis=1)
    print('Success: E transformed from Z')
    return E

def update_C(C, Z):
    k = C.shape[0]
    for topic in range(k):
        C[topic, :] = np.sum(Z == topic, axis=2).sum(axis=0)
    print('Success: C transformed from Z')
    return C

def update_B(B, C):
    # Note this is the transformation from C
    for topic in range(0, len(B)):
        B[topic] = C[topic] / sum(C[topic])
    print('Success: B transformed from C')
    return B

def update_Sigma(K):
    Sigma = np.linalg.inv(K)
    print('Success: Sigma transformed from K')
    return Sigma

In [5]:
# Random / Generating functions

def build_topic_distribution(seed):
    np.random.seed(seed)
    distribution = np.random.random(V)
    return distribution / distribution.sum()

def sample_B(seed):
    # B is the matrix whose rows are the distribution of topic i over the vocabulary
    # Each row means : for each topic i we have the probability of word i to occur
    # TODO: Change with Dirichlet prior -> See line to change
    b = np.empty((k,V))
    np.random.seed(seed)
    for i in range(k):
        b[i,:] = build_topic_distribution(seed)  # TODO: Change
    return b

def sample_G(k, gamma, seed):  # Won't update Sigma automatically anymore
    # Bernoulli for G
    # generate a random adjacency matrix
    np.random.seed(seed)
    matrix = np.array([[int(bernoulli.rvs(p=gamma, size=1)) for i in range(k)] for j in range(k)])
    for i in range(k):
        matrix[i][i] = 0
    for i in range(k):
        for j in range(k):
            matrix[j][i] = matrix[i][j]
    return matrix

def sample_K(k, seed):  # Won't update Sigma automatically anymore
    # I can build K for using make_sparse_spd_matrix from sklearn.datasets for example
    np.random.seed(seed)
    K = make_sparse_spd_matrix(k, alpha=0.95, norm_diag=False, smallest_coef=0.1, largest_coef=0.9, random_state=None)
    return K

def sample_H(Sigma, D, k, seed):  # Won't update Theta automatically anymore
    # Multivariate Normal
    mu = np.zeros(k)
    np.random.seed(seed)
    H = np.random.multivariate_normal(mu, Sigma, D)
    return H

def sample_Z_from_W(W, k, seed):
    D, V = W.shape
    M = int(W.max())
    Z = -np.ones((D, V, M))
    np.random.seed(seed)
    for d in range(D):
        for w in range(V): 
            occurrences = W[d, w]
            Z[d, w, 0:occurrences] = np.random.randint(0, k, size=occurrences)    
    return Z

In [6]:
# Main Simulator Class
class Simulator:
    
    # Remember we will have indexes starting from 0 so all max are -=1
    
    def __init__(self, D, V, M, k, gamma, seed):
        # Create zero matrices for all possible matrices
        self.W = np.zeros((D, V))  # matrix of D×V where Wdn is counter of appearances of the word n in document d
        self.B = np.zeros((k, V))  # matrix of kxV where Bz is the parameter vector of the distribution for the z-th topic
        self.C = np.zeros((k, V))  # matrix of kxV where Cz is the count vec of sampled topics over each word for all docs
        self.E = np.zeros((D, k))  # matrix of Dxk where Ed is the count vec of sampled drawings for topic z over all words for each doc
        self.H = np.zeros((D, k))  # H_d is eta_d
        self.Theta = np.zeros((D, k))  # This is just a transformation of H
        self.G = np.zeros((k, k))  # Adjacency Matrix (Check also python package "networkx" for graph objects!)
        self.K = np.zeros((k, k))  # Precision matrix of G
        self.Sigma = np.zeros((k, k))  # Inverse of K
        self.Z = None  # Z shouldn't be created before W is sampled
        self.D = D
        self.V = V
        self.M = M
        self.k = k
        self.gamma = gamma
        self.seed = seed  # Random seed
        
    def get_M(self):
        # Ref: https://numpy.org/doc/stable/reference/generated/numpy.matrix.max.html
        return int(self.W.max())
    
    # Generations
    def generate_WZ(self):
        if M == 0:
            raise Exception('Error: Input 0 for the chosen method')
        elif np.sum(self.Theta, axis=1).sum(axis=0) == 0:
            raise Exception('Error: Theta matrix 0')
        elif np.sum(self.B, axis=1).sum(axis=0) == 0:
            raise Exception('Error: B matrix 0')
        
        Z = [[[] for k in range(self.V)] for j in range(self.D)]  # Unknown amount of repetitions
        np.random.seed(self.seed)
        # Ref https://numpy.org/doc/stable/reference/random/generated/numpy.random.multinomial.html
        # Multinomial drawing for Z and then W
        for d in range(self.D):
            N_d = np.random.randint(0, self.M * self.V)  # Maximum of words in the document
            for n in range(N_d):
                
                # Multinomial drawing from Theta, because it has to be normalized
                # This will give a canonical vector over k
                mult = np.random.multinomial(1, self.Theta[d], size=1)  # This is a vector of 0's with a single 1
                z = np.argmax(mult)  # This is the index of the 1 (Topic index)
                
                # Multinomial drawing from Beta
                # This will give a canonical vector over V
                mult = np.random.multinomial(1, self.B[z], size=1)  # This is a vector of 0's with a single 1
                w = np.argmax(mult)  # This is the index of the 1 (Word index)
                
                Z[d][w].append(z)
                self.W[d,w] += 1
        
        print('Success: W generated')
        self.Z = - np.ones((self.D, self.V, self.get_M()))
        for d in range(self.D):
            for n in range(self.V):
                for i in range(len(Z[d][n])):  # Only take existing topics
                    self.Z[d][n][i] = Z[d][n][i]  # Replace
        print('Success: Z generated')
    
    # Transformations
    def update_Theta(self):
        self.Theta = update_Theta(self.Theta, self.H)
    
    def update_E(self):
        self.E = update_E(self.E, self.Z)
    
    def update_C(self):
        self.C = update_C(self.C, self.Z)
    
    def update_Sigma(self):
        self.Sigma = update_Sigma(self.K)
    
    # Initializing with real data
    # def save_W()
    
    # Priors
    def sample_B(self):
        self.B = sample_B(self.seed)
        
    def sample_GK(self):  # Here we can update Sigma automatically
        self.G = sample_G(self.k, self.gamma, self.seed)
        self.K = sample_K(self.k, self.seed)
        self.update_Sigma()
    
    def sample_H(self):  # Here we can update Theta automatically
        self.H = sample_H(self.Sigma, self.D, self.k, self.seed)
        self.update_Theta()
    
    def generate_all_data(self):
        # TODO: This should run all relevant methods one after the other in order to fully populate all data matrixes
        self.sample_B()  # Will get B
        self.sample_GK()  # Will get G, K, Sigma
        self.sample_H()  # Will get H, Theta from Sigma
        self.generate_WZ()  # Will get W, Z from Theta, B
        self.update_E()  # Will get E from Z
        self.update_C()  # Will get C from Z
        pass

# 2 MC SAMPLER

# 2.1 MCMC Sampling

### Task:

You must implement a function that receives matrices $W$, $\Theta_{i+1}$ and $B_i$ and generates the next $Z_{i+1}$ and $B_{i+1}$.

In [7]:
def binary_search(sequence, item):
    begin_index = 0
    end_index = len(sequence)-1
    
    if sequence[begin_index] <= item and item <= sequence[end_index]:
        while begin_index < end_index - 1:  # Finish when the list has 2 items: Begin and end
            midpoint = (end_index + begin_index)//2
            midpoint_value = sequence[midpoint]
            if midpoint_value < item:
                begin_index = midpoint
            else:
                end_index = midpoint
        if sequence[begin_index] == item:
            return begin_index + 1
        elif item <= sequence[end_index]:
            return end_index
    else:
        return -1

In [8]:
def MC_sample_Z(Z, W, Theta, B, E, C):  # D, k are global variables
    for d in range(D):
        for v in range(V):
            I_di = int(W[d][v])
            for j in range(I_di):
                z_hat = int(Z[d][v][j])
                
                E[d][z_hat] = max(0, E[d][z_hat]-1)
                
                C[z_hat][v] = max(0, C[z_hat][v]-1)
                
                Rho = []  # Needs to start from zero to have the interval to fall into topic 1
                Rho_z = 0
                Rho.append(Rho_z)
                
                for z in range(k):
                    # Compute the denominator sum
                    C_vk = 0
                    for b in range(V):
                        if b != v:
                            C_vk += C[z][b]
                    # Compute the upper limits of the topic probabilities
                    d_part = E[d][z] + Theta[d][z]
                    z_part = C[z][v] + B[z][v]
                    denom = C_vk + V*B[z][v]
                    Rho_z += d_part * z_part / denom
                    Rho.append(Rho_z)
                
                u = np.random.uniform(0, Rho[-1])
                z_hat = binary_search(Rho, u) - 1
                E[d][z_hat] += 1
                C[z_hat][v] += 1
                Z[d][v][j] = z_hat
    # Note that we directly modify Z since the update per topic helps for the next iteration 
    return Z

# 2.2 Beta sampling

### Task:

You must implement a function that receives matrices $C_i$, $W$ and vector $\alpha$ and generates the next $B_{i+1}$.

In [9]:
def MC_sample_B():
    # Sample B from Dir posterior
    return B

# 2.3 Metropolis-Hastings MC Sampling

### Task:

You must implement a function that receives matrices $E_i$, $K_i$ and vector $\mu$ and generates the next $H_{i+1}$.


. $E$ matrix of $D \times k$ where $E_d$ is the $k$-dim vector of counts of sampled drawings for the $z$-th topic over all words for each document

. $K$ matrix of $k \times k$ representing the precision matrix associated to the graph $G$

. $\mu = 0$

. $H$ matrix of $D \times k$ where $H_d = \eta_d$ is the $k$-dim vector of the topic prevalences over document $d$

In [10]:
import numpy as np
import numpy.linalg

In [11]:
# np.seterr(all='raise')  # Will raise error instead of warning

In [12]:
def sampled_distribution_kernel(eta, K, E):
    k = eta.shape[0]
    eta_K_eta = -0.5 * eta.dot(K.dot(eta))
    E_eta = E.dot(eta)
    sum_eta_pow_k = np.sum(np.exp(eta)) ** k
    return np.exp(eta_K_eta + E_eta) / sum_eta_pow_k  # This np.exp raises a warning when  eta_K_eta + E_eta > 706

In [13]:
def MC_sample_H(E, Sigma, H_current=None, burn_in=100, seed=None):
    
    np.random.seed(seed)
    
    K = np.linalg.inv(Sigma)
    
    D, k = E.shape  # Number of documents, Number of topics
    
    if H_current is None:
        H_current = np.zeros((D, k))
    
    H_sampled = np.zeros((D, k))
    
    for d in range(D):  # Iterating over each document
        current_eta = H_current[d]
        E_d = E[d]
        for iteration in range(burn_in + 1):
            
            # Sampling proposed eta from multivariate normal (q "proposal density")
            proposed_eta = np.random.multivariate_normal(current_eta, Sigma)
            
            # Compute acceptance probability
            p_proposed_eta = sampled_distribution_kernel(proposed_eta, K, E_d)
            p_current_eta = sampled_distribution_kernel(current_eta, K, E_d)
            if p_proposed_eta == np.inf or p_current_eta == 0:  # Avoiding divide by 0 and other numerical creeps
                alpha = 1
            else:
                alpha = min(1, p_proposed_eta / p_current_eta)
            
            if alpha == 1 or np.random.uniform(0.0, 1.0) < alpha:
                current_eta = proposed_eta
            
        H_sampled[d] = current_eta
    
    return H_sampled

# 2.4 BDMCMC Sampling

### Task:

You must implement a function that receives matrices $W$, $Z_{i+1}$ and $H_{i+1}$ and generates the next $G_{i+1}$ and $K_{i+1}$.

In [14]:
import subprocess

# Remember to install BDgraph package version 2.62 on your R environment
# You'll need to run the following commands:
# remove.packages("BDgraph")
# install.packages("remotes")
# library(remotes)
# install_version("BDgraph", "2.62")


def MC_sample_K(G, b, shape_matrix, n_of_samples):
    # save G to csv
    np.savetxt("adj.csv", G, delimiter=",")
    # save shape_matrix to csv (OR pass them as parameters to RScript)
    np.savetxt("shape.csv", shape_matrix, delimiter=",")
    # call R script using python.subprocess
    result = subprocess.run(f"Rscript --vanilla rgwish.R {b} {n_of_samples}", shell=True) # -> HOW MANY SAMPLES ARE REQUIRED? SHOULD THIS ALSO BE VARIABLE AND THEREFORE PASSED TO R?
    if result.returncode != 0:
        raise Exception("Rscript error! Check the previous logs for more details")

    # read the results from csv (OR get the result back from R script)
    
    # Always better to open file handlers within a "with" statement (otherwise you must take care of closing the file)
    with open("gwish.csv", "r") as csv_file:  
        K = np.loadtxt(csv_file, delimiter=",")
    
    return K

In [15]:
#This is a test of MC_sample_K
G = [[0,0,1,0,0],
    [0,0,0,0,0],
    [1,0,0,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0]]
shape_matrix = np.eye(5, dtype=int)
MC_sample_K(G, 5, shape_matrix, n_of_samples=1)

[1] "[R] rgwish will now be sampling 1 samples. The b (degrees of freedom) is set to 5"
[1] "[R] rgwish succesfully executed!"


array([[ 1.45,  0.  ,  0.5 ,  0.  ,  0.  ],
       [ 0.  ,  8.62,  0.  ,  0.  ,  0.  ],
       [ 0.5 ,  0.  ,  1.38,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  8.91,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  , 13.11]])

In [16]:
def MC_sample_G(W, Z, H, K, G, E):

    N = G.shape[0]
    delta_K = 0
    beta_K = 0

    death_rates = np.zeros((k,k))
    birth_rates = np.zeros((k,k))
    
    n = D  # It's either of the shape indices of H
    # from the paper I'm not sure yet which but looks the docs for us
    
    S = np.eye(k)  # prior parameter -> Caspar has it as shape_matrix
    b = k - 1  # prior parameter -> Caspar has it below

    PrHK = lambda K, H:        K.size ** (n / 2) * np.exp(-0.5 * np.trace(np.matmul(np.matmul(K, H.T), H)))
    PrK_G = lambda K, G, D, b: K.size ** (b + D - 2) * np.exp(-0.5 * np.trace(np.matmul(S + np.matmul(H.T, H), K)))
    PrG = lambda gamma, E:  (gamma / (1 - gamma)) ** (E.size)
    PrKG_H = lambda K, G, H, D, b, gamma, E: PrHK(K, H) * PrK_G(K, G, D, b) * PrG(gamma, E)

    Pr_init = PrKG_H(K, G, H, D, k - 1, gamma, E)

    for i in range(N):
        for j in range(i + 1, N):
            if G[i,j]:
                G_loop = G.copy()
                G_loop[i,j] = 0
                G_loop[j,i] = 0
                #technically, we should compute K_loop here...
                Pr_loop = PrKG_H(K,G_loop,H,D,b,gamma,E)

                death_rate = Pr_loop / Pr_init

                if death_rate > 1:
                    death_rate = 1
                death_rates[i,j] = death_rate
                death_rates[j,i] = death_rate
                delta_K += death_rate

            else:
                G_loop = G.copy()
                G_loop[i,j] = 1
                G_loop[j,i] = 1
                #technically, we should compute K_loop here...
                Pr_loop = PrKG_H(K, G_loop, H, D, b, gamma, E) 

                birth_rate = Pr_loop / Pr_init

                if birth_rate > 1:
                    birth_rate = 1
                birth_rates[i,j] = birth_rate
                birth_rates[j,i] = birth_rate
                beta_K += birth_rate
    
    W = 1/(beta_K + delta_K)

    pr_death = W * death_rates
    pr_birth = W * birth_rates

    G[pr_death > 0.5] = 0
    G[pr_birth > 0.5] = 1
   
    return G

# TESTING

### Data Generation

In [17]:
test0 = Simulator(D, V, M, k, gamma, seed=1996)
test0.sample_GK()  # Will get G, K, Sigma
test0.sample_H()  # Will get H, Theta from Sigma
test0.sample_B()  # Will get B
test0.generate_WZ()
test0.update_E()

# Note for debugging:
# After updating the class, you need to re run the class object initialization
# Else for some reason it uses the outdated code

# K:
# (well yeah ;) jupyter is an interactive programming interface
# it only runs the code when you tell it to do so.
# If you define class C, then create object c = C(), then redefine class C, 
# object c will still be using the old definition of C)

Success: Sigma transformed from K
Success: Theta transformed from H
Success: W generated
Success: Z generated
Success: E transformed from Z


In [18]:
test0.W  # Sadly it's not really respected that M is the real max, just a desired one

array([[15.,  1., 10., ...,  9.,  9.,  8.],
       [ 6.,  1.,  7., ...,  4.,  2.,  3.],
       [ 5.,  0.,  3., ...,  6.,  4.,  0.],
       ...,
       [13.,  2.,  7., ...,  8., 13.,  4.],
       [ 5.,  0.,  8., ..., 12.,  3.,  2.],
       [10.,  2.,  9., ...,  8.,  4.,  6.]])

In [19]:
# It would be nice to have it all in one function
test1 = Simulator(D, V, M, k, gamma, seed=1979)
test1.generate_all_data()
test1.W

Success: Sigma transformed from K
Success: Theta transformed from H
Success: W generated
Success: Z generated
Success: E transformed from Z
Success: C transformed from Z


array([[ 1.,  0.,  0., ...,  2.,  1.,  1.],
       [19., 11.,  7., ..., 17., 10., 11.],
       [ 5.,  4.,  3., ...,  2.,  5.,  5.],
       ...,
       [ 6.,  5.,  3., ...,  4.,  2.,  3.],
       [ 2.,  2.,  1., ...,  1.,  1.,  2.],
       [ 5.,  3.,  2., ...,  3.,  0.,  1.]])

In [20]:
# It would be nice to have it all in one function
test2 = Simulator(D, V, M, k, gamma, seed=1979)
test2.generate_all_data()
assert np.all(np.equal(test1.W, test2.W))  # Confirming that with the same seed, data generated are the same

Success: Sigma transformed from K
Success: Theta transformed from H
Success: W generated
Success: Z generated
Success: E transformed from Z
Success: C transformed from Z


### MC Samplers

In [21]:
MC_sample_Z(test0.Z, test0.W, test0.Theta, test0.B, test0.E, test0.C)

array([[[ 2.,  2.,  2., ..., -1., -1., -1.],
        [ 4., -1., -1., ..., -1., -1., -1.],
        [ 3.,  3.,  3., ..., -1., -1., -1.],
        ...,
        [ 8.,  8.,  8., ..., -1., -1., -1.],
        [ 6.,  6.,  6., ..., -1., -1., -1.],
        [ 2.,  9.,  1., ..., -1., -1., -1.]],

       [[ 2.,  2.,  2., ..., -1., -1., -1.],
        [ 4., -1., -1., ..., -1., -1., -1.],
        [ 3.,  3.,  3., ..., -1., -1., -1.],
        ...,
        [ 2.,  2.,  8., ..., -1., -1., -1.],
        [ 6.,  6., -1., ..., -1., -1., -1.],
        [ 9.,  1.,  9., ..., -1., -1., -1.]],

       [[ 2.,  2.,  2., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [ 3.,  3.,  3., ..., -1., -1., -1.],
        ...,
        [ 8.,  2.,  8., ..., -1., -1., -1.],
        [ 6.,  6.,  6., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]],

       ...,

       [[ 2.,  2.,  2., ..., -1., -1., -1.],
        [ 4.,  4., -1., ..., -1., -1., -1.],
        [ 3.,  3.,  3., ..., -1., -1., -1.

In [22]:
MC_sample_H(test0.E, test0.Sigma, seed=1984)

/tmp/ipykernel_76815/2545393832.py:6: RuntimeWarning: overflow encountered in exp
  return np.exp(eta_K_eta + E_eta) / sum_eta_pow_k  # This np.exp raises a warning when  eta_K_eta + E_eta > 706


array([[ 7.85412481e+00,  1.26706535e+01,  8.26109541e+00,
         6.21650839e+00,  1.20956802e+01,  1.62024347e+01,
         1.47641876e+01,  6.07073441e+00, -4.16293424e+00,
         1.39932423e+00],
       [ 1.17212103e+00,  1.69643157e+01,  6.91127625e+00,
         1.98969042e+01,  2.36275282e+01,  1.17196238e+01,
         1.16839656e+01,  5.70133869e+00,  7.50269627e+00,
         4.55039709e+00],
       [ 2.47455887e+00,  8.51351464e+00,  9.43350254e+00,
         1.24299722e+01,  1.00452533e+01,  6.49852201e+00,
         9.16184225e+00,  1.01007328e+00,  1.36798804e+01,
         1.13720506e+01],
       [ 1.07733717e+01,  8.65711803e+00,  1.00689680e+01,
         8.33072476e+00,  1.22701849e+01,  1.03539771e+01,
         7.04235285e+00,  3.82191409e+00,  3.76853220e+00,
         1.05589352e+01],
       [ 9.42809637e+00,  8.51292373e+00,  6.25657723e+00,
         1.24557757e+01, -3.11145738e+00,  1.16953417e+01,
         1.10566128e+01,  1.01155001e+01,  1.54194663e+00,
         2.

In [23]:
MC_sample_G(test0.W, test0.Z, test0.H, test0.K, test0.G, test0.E)

/tmp/ipykernel_76815/30321175.py:47: RuntimeWarning: invalid value encountered in double_scalars
  birth_rate = Pr_loop / Pr_init
/tmp/ipykernel_76815/30321175.py:32: RuntimeWarning: invalid value encountered in double_scalars
  death_rate = Pr_loop / Pr_init


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [24]:
#This is a test of MC_sample_K
G = [[0,0,1,0,0],
    [0,0,0,0,0],
    [1,0,0,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0]]
shape_matrix = np.eye(5, dtype=int)
MC_sample_K(G, 5, shape_matrix, n_of_samples=1)

[1] "[R] rgwish will now be sampling 1 samples. The b (degrees of freedom) is set to 5"
[1] "[R] rgwish succesfully executed!"


array([[ 7.67,  0.  , -2.83,  0.  ,  0.  ],
       [ 0.  ,  2.13,  0.  ,  0.  ,  0.  ],
       [-2.83,  0.  ,  2.76,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  4.02,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  6.95]])

# MAIN ALGORITHM

### Generating Data

In [25]:
simulated_data = Simulator(D, V, M, k, gamma, seed=1984)
simulated_data.generate_all_data()

Success: Sigma transformed from K
Success: Theta transformed from H
Success: W generated
Success: Z generated
Success: E transformed from Z
Success: C transformed from Z


In [26]:
# Input Data:
simulated_data.W

array([[ 0.,  5.,  0., ..., 11., 10.,  2.],
       [ 0.,  0.,  0., ...,  0.,  4.,  0.],
       [ 0.,  3.,  0., ...,  1.,  6.,  1.],
       ...,
       [ 0.,  3.,  0., ..., 12., 10.,  3.],
       [ 0.,  7.,  0., ..., 11., 10.,  2.],
       [ 0.,  1.,  0., ...,  5.,  4.,  0.]])

# SAMPLER

In [27]:
# # Initial guesses
initial = Simulator(D, V, M, k, gamma, 2020)
initial.sample_GK()
initial.sample_B()
initial.sample_H()

Success: Sigma transformed from K
Success: Theta transformed from H


In [29]:
max_iteration = 100

# Data
W = simulated_data.W

# Initialization
Sigma = initial.Sigma
K = np.linalg.inv(Sigma)
B = initial.B
Theta = initial.Theta
G = initial.G

shape_matrix = np.eye(k)

Z = sample_Z_from_W(simulated_data.W.astype(int), k, 2020)  # Random sample of Z based on actual data

E = update_E(np.zeros((D, k)), Z)  # Transformation of Z
C = update_C(np.zeros((k, V)), Z)  # Transformation of Z

b = k - 1


for iteration in range(max_iteration):
    
    # Step 1
    Z = MC_sample_Z(Z, W, Theta, B, E, C)
    E = update_E(E, Z)  # get E from Z
    C = update_C(C, Z)  # get C from Z ...
    
    # Step 2
    H = MC_sample_H(E, Sigma)
    Theta = update_Theta(Theta, H)  # get Theta from H
    
    
    # Step 3
    G_new = MC_sample_G(W, Z, H, K, G, E)
    if np.all(G_new == G):
        print('G matrix has NOT changed!')
    G = G_new
    
    # Need to define b and shape_matrix!
    K = MC_sample_K(G, b, shape_matrix, n_of_samples=1)
    Sigma = np.linalg.inv(K)
    
# #     shape_matrix =
# #     b =
    
    # Hope for convergence!
    wrong_edges = np.sum(G != simulated_data.G)
    error = np.linalg.norm(Sigma - simulated_data.Sigma)
    print(f"At iteration {iteration}, the wrong edges are {wrong_edges} and the error on Sigma is {error}")

Success: E transformed from Z
Success: C transformed from Z
Success: E transformed from Z
Success: C transformed from Z


/tmp/ipykernel_76815/2545393832.py:6: RuntimeWarning: overflow encountered in exp
  return np.exp(eta_K_eta + E_eta) / sum_eta_pow_k  # This np.exp raises a warning when  eta_K_eta + E_eta > 706


Success: Theta transformed from H
G matrix has NOT changed!
[1] "[R] rgwish will now be sampling 1 samples. The b (degrees of freedom) is set to 9"
[1] "[R] rgwish succesfully executed!"
At iteration 0, the wrong edges are 2 and the error on Sigma is 3.3752120375801216


/tmp/ipykernel_76815/30321175.py:47: RuntimeWarning: invalid value encountered in double_scalars
  birth_rate = Pr_loop / Pr_init
/tmp/ipykernel_76815/30321175.py:32: RuntimeWarning: invalid value encountered in double_scalars
  death_rate = Pr_loop / Pr_init


Success: E transformed from Z
Success: C transformed from Z
Success: Theta transformed from H
G matrix has NOT changed!
[1] "[R] rgwish will now be sampling 1 samples. The b (degrees of freedom) is set to 9"
[1] "[R] rgwish succesfully executed!"
At iteration 1, the wrong edges are 2 and the error on Sigma is 3.4291704820291042
Success: E transformed from Z
Success: C transformed from Z
Success: Theta transformed from H
G matrix has NOT changed!
[1] "[R] rgwish will now be sampling 1 samples. The b (degrees of freedom) is set to 9"
[1] "[R] rgwish succesfully executed!"
At iteration 2, the wrong edges are 2 and the error on Sigma is 3.3664640849823764
Success: E transformed from Z
Success: C transformed from Z
Success: Theta transformed from H
G matrix has NOT changed!
[1] "[R] rgwish will now be sampling 1 samples. The b (degrees of freedom) is set to 9"
[1] "[R] rgwish succesfully executed!"
At iteration 3, the wrong edges are 2 and the error on Sigma is 3.221707205490798
Success: E 

KeyboardInterrupt: 